In [53]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import re
import collections
from datetime import datetime

In [9]:
root = 'http://www.public-affairs.ch'
member_url = 'http://www.public-affairs.ch/de/ueber-uns/mitglieder'
print (member_url)

http://www.public-affairs.ch/de/ueber-uns/mitglieder


### some funtions to make it easier

In [4]:
def souper(page_url):
    """beautifulsoup page reader"""
    user_agent = 'Mozilla/4.0 (compatible; MSIE 6.1; Windows XP)'
    headers = { 'User-Agent' : user_agent }
    page = get(page_url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser') 
    return soup

In [5]:
def all_member_pages(url):
    """all pages that contains member info"""
    page_urls = []
    page_urls.append(url)
    soup = souper(url)
    pagerbox = soup.find('ul', class_='pager')
    pages = pagerbox.findAll('li', class_='pager-item')
    for p in pages:
        url = p.find('a').get('href')
        url = root+url
        page_urls.append(url)
    return page_urls

In [6]:
def boxes(pages_list):
    """this gets all the basic info from the overview pages on the 'mitglieder' pages"""
    people = []
    for pagepath in pages_list:  
        soup = souper(pagepath)
        table = soup.find('table', class_='views-view-grid cols-4')
        td = table.findAll('td')
        for content in td:
            url = content.find('a').get('href')
            name = content.find('a').find('h2').text
            company = content.find('div',class_='views-field views-field-field-mitglied-unternehmen').find('div', class_='field-content').text
            function = content.find('div',class_='views-field views-field-field-mitglied-funktion').find('div', class_='field-content').text
            person = {'url':url,'name': name, 'company':company, 'function':function}
            people.append(person)
    return people

In [7]:
def text_prepare(str):
    """this cleans te text from the memberpage"""
    filter = {'views'      : ''  ,
              'field'      : ''  ,
              'mitglied'   : ''  ,
              'conditional': ''  ,
              '-'          : ' ' ,
              '\n'         : ', ',
              ' [at] '     : '@'  }
    for i in filter:
        str = str.replace(i,filter[i])
        str = str.lstrip()
        str = str.rstrip()
    return str

In [8]:
def member_details(url):
    """this gets all the info from the memberpage if possible it gets the 
    fieldname from the div-class"""
    soup = souper(url)
    block = soup.find('div', id="content", role="main")
    div = block.findAll('div', class_='views-field')
    result_dict = {}
    for i in div:
        fieldname = i.get('class')[1]
        fieldname = text_prepare(fieldname)
        text = text_prepare(i.text)
        try:
            split = text.split(': ')
            text = split[1]
            fieldname = split[0]
        except:
            pass
        result_dict[fieldname] = text
    return(result_dict)

### the business

#### first get all the members from the mitglied' pages. This can already be used for who is a member

In [26]:
memberpages = all_member_pages(member_url)
list_of_people = boxes(memberpages)
members = pd.DataFrame(list_of_people)
members = members[['name','company','function','url']]
members.head()

,name,company,function,url
0,Laurens Abu-Talib,usic,Leiter Politik,/de/ueber-uns/mitglieder/mitglied/3029
1,Chantal Aeby Pürro,Schweizerischer Weinbauernverband,Geschäftsführerin,/de/ueber-uns/mitglieder/mitglied/2519
2,Dr. Paul Aenishänslin,Paul Aenishaenslin Public Affairs & Communicat...,Geschäftsführer,/de/ueber-uns/mitglieder/mitglied/2520
3,Regina Ammann,Syngenta,Leiterin External und Public Affairs Schweiz,/de/ueber-uns/mitglieder/mitglied/2521
4,Fredj Ariche,ARICHE PROTECTION & DEFENSE SECURITY SOLUTIONS,,/de/ueber-uns/mitglieder/mitglied/3295


### get all the details from every specific memberpage

In [27]:
#create a list of urls to scrape
member_urls = members['url'].tolist()

In [10]:
#example of data retrieved from a memberpage
member_details('http://www.public-affairs.ch/de/ueber-uns/mitglieder/mitglied/3194?page=4')

{'1': 'Inhaberin',
 '10': 'Consulting',
 '2': 'Wallstrasse 13',
 '3': '4051 Basel',
 '7': 'katrin.schweren@drschwerenconsulting.ch',
 '8': 'www.drschwerenconsulting.ch',
 'Arbeitgeber': ' Dr. Schweren Consulting',
 'Auftraggeber': ' Swisscom Energy Solutions, BX Swiss, IAF (abgeschlossen)',
 'Curia Vista Sachgebiete': ' Politischer Rahmen, Parlament, Internationale Politik, Sicherheitspolitik, Europapolitik, Recht, Wirtschaft, Finanzwesen, Soziale Fragen, Medien/Kommunikation, Verkehr, Umwelt, Energie, Internationale Organisationen',
 'Telefon Zentrale': ' 0041 61 271 27 11',
 'Telefon direkt': ' 0041 79 345 26 39',
 'name': 'Dr. Katrin Schweren',
 'unternehmen': 'Dr. Schweren Consulting'}

In [28]:
# this scrapes all the details from every page 
# and puts it in a dataframe for easy processing
r = []
for i in member_urls:
    r.append(member_details(root+i))
member_details = pd.DataFrame(r)
member_details.head(1)

,1,10,11,2,3,7,8,Arbeitgeber,Auftraggeber,Curia Vista Sachgebiete,Fax,Handy,Mitglied seit,Telefon Zentrale,Telefon direkt,Weitere Interessenbindungen,auftraggeber,interessenb,name,unternehmen
0,Leiter Politik,Geschäftsstelle,NaN,Effingerstrasse 1,3001 Bern,laurens.abu talib@usic.ch,"www.usic.ch, www.kellerhals carrard.ch, www.af...",Kellerhals Carrard,Schweizerische Vereinigung beratender Ingenie...,"Politischer Rahmen, Parlament, Recht, Wirtsch...",NaN,NaN,2015,+41 31 970 08 88,NaN,Mitglied FDP.Die Liberalen Zürich 7+8,NaN,NaN,Laurens Abu Talib,usic


In [30]:
# cleanup of column names
column_names = {'1':'Position','10':'dept','11':'postfach','2':'Address', '3':'PostalCode_Place',
         '7':'Email','8':'urls'}
member_details.rename(columns=column_names,inplace=True)
member_details.dropna(axis=1,how='all') #deletes empty columns

,Position,dept,postfach,Address,PostalCode_Place,Email,urls,Arbeitgeber,Auftraggeber,Curia Vista Sachgebiete,Fax,Handy,Mitglied seit,Telefon Zentrale,Telefon direkt,Weitere Interessenbindungen,auftraggeber,interessenb,name,unternehmen
0,Leiter Politik,Geschäftsstelle,NaN,Effingerstrasse 1,3001 Bern,laurens.abu talib@usic.ch,"www.usic.ch, www.kellerhals carrard.ch, www.af...",Kellerhals Carrard,Schweizerische Vereinigung beratender Ingenie...,"Politischer Rahmen, Parlament, Recht, Wirtsch...",NaN,NaN,2015,+41 31 970 08 88,NaN,Mitglied FDP.Die Liberalen Zürich 7+8,NaN,NaN,Laurens Abu Talib,usic
1,Geschäftsführerin,NaN,NaN,Belpstrasse 26,3007 Bern,chantal.aeby@fsv.ch,,Schweizerischer Weinbauernverband,NaN,keine Informationen,+41(031) 398 52 61,NaN,NaN,+41 (031) 398 52 60,NaN,NaN,NaN,NaN,Chantal Aeby Pürro,Schweizerischer Weinbauernverband
2,Geschäftsführer,NaN,Postfach 113,Handschinweg 1,4460 Gelterkinden,paul.aenishaenslin@bluewin.ch,,Paul Aenishaenslin Public Affairs & Communica...,"Carbura, Zürich (Public Affairs & Kommunikati...","Parlament, Sicherheitspolitik, Europapolitik,...",+41 61 981 68 69,NaN,2003,+41 61 989 68 58,NaN,NaN,NaN,NaN,Dr. Paul Aenishänslin,Paul Aenishaenslin Public Affairs & Communicat...
3,Leiterin External und Public Affairs Schweiz,Business Sustainability,NaN,Schwarzwaldallee 215,4058 Basel,regina.ammann@syngenta.com,,"Syngenta , Basel","Syngenta , Basel","Politischer Rahmen, Internationale Politik, W...",+41 61 323 53 31,NaN,NaN,+41 61 323 09 87,+41 79 336 95 66,Schweizerisches FAO Komitee (ausserparlamenta...,NaN,NaN,Regina Ammann,Syngenta
4,NaN,NaN,NaN,Loretohöhe 7,6300 Zug,defendo75@me.com,,ARICHE PROTECTION & DEFENSE SECURITY SOLUTIONS,NaN,"Politischer Rahmen, Parlament, Internationale...",NaN,NaN,NaN,076 793 4798,NaN,NaN,NaN,NaN,Fredj Ariche,ARICHE PROTECTION & DEFENSE SECURITY SOLUTIONS
5,Director Government Affairs,Government Affairs,NaN,NaN,8304 Wallisellen,juerg.aschwanden@gmail.com,,Liberty Global,NaN,Medien/Kommunikation,NaN,NaN,13.12.2011,NaN,NaN,NaN,NaN,NaN,Jürg Aschwanden,Liberty Global
6,Public Affairs,Rechtsdienst,Postfach 9870,Stauffacherstrasse 101,8036 Zürich,dominik.bachmann@aids.ch,"www.aids.ch, www.knnr.ch","Dominik Bachmann Herrenausstatter, Aids Hilfe...",Aids Hilfe Schweiz,"Recht, Soziale Fragen, Kultur und Religion, G...",NaN,NaN,NaN,+41 44 447 11 11,+41 76 576 36 64,Network Gay Leadership,NaN,NaN,Dominik Bachmann,Aids Hilfe Schweiz
7,Berater,NaN,NaN,Rte du Bugnon 47,1752 Villars sur Glâne,othmar.baeriswyl@hslu.ch,www.mediata.ch,Hochschule Luzern,mediata sa,"Raumplanung/Wohnungswesen, Bildung, Medien/Ko...",NaN,+41 79 136 85 28,2000,NaN,NaN,NaN,NaN,NaN,Dr. Othmar Baeriswyl,Hochschule Luzern
8,"Manager, Public Policy",NaN,NaN,Stegstrasse 19,8808 Pfäffikon SZ,roberto.balmer@outlook.com,www.linkedin.com,ABCgate GmbH,ABCgate GmbH,"Politischer Rahmen, Parlament, Sicherheitspol...",NaN,NaN,NaN,+41798828351,NaN,NaN,NaN,NaN,Roberto Balmer,ABCgate GmbH
9,Geschäftsführer,NaN,NaN,Konolfingenstrasse 26,3510 Häutligen,mb@bangerter.partners,"www.assgp.ch, www.vitagate.ch, www.drogistenve...",bangerter.partners gmbh,Schweizerischer Fachverband für Selbstmedikat...,"Parlament, Gesundheit, Bildung",NaN,+41 79 455 74 90,NaN,NaN,+41 31 791 31 36,"Dachverband Komplementärmedizin Dakomed, Co P...",NaN,NaN,Martin Bangerter,bangerter.partners gmbh


In [ ]:
now = datetime.strftime(datetime.now(), '%y%m%d_%H%m')
member_details.to_csv(now +'_public-affairs-members.csv', sep=';')
